In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

# Table name and location
LakehouseName = 'LH_CatManUsage'
DeltaTableName = "Date"


# Create pandas dataframe
start_date = "2024-01-01"
end_date = "2030-12-31"


StatementMeta(, cf4561ed-bff4-4185-a0fc-ed0a34b91db6, 3, Finished, Available, Finished)

In [2]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset, BMonthEnd
from datetime import datetime

StatementMeta(, cf4561ed-bff4-4185-a0fc-ed0a34b91db6, 4, Finished, Available, Finished)

In [3]:
def date_dimension(start_date: str, end_date: str) -> pd.DataFrame:
    try:
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Invalid date format. Please provide dates in 'yyyy-mm-dd' format.")

    if end_date <= start_date:
        raise ValueError("End date should be after start date.")

    df = pd.DataFrame({"Date": pd.date_range(start_date, end_date)})

    today = pd.Timestamp.now().normalize()
    current_week = today.week
    current_month = today.month
    current_quarter = today.quarter
    current_year = today.year

    # df["DateKey"] = df.Date.dt.strftime('%Y%m%d').astype(int)
    # df["ISODateName"] = df.Date.dt.strftime('%Y-%m-%d')

# DESCRIPTIONS
    df["Weekday"] = df.Date.dt.day_name()
    df["Weekday_Short"] = df.Date.dt.day_name().str[:3]
    df["Month_Name"] = df.Date.dt.month_name()
    df["Month_Short"] = df.Date.dt.month_name().str[:3]


# CALENDAR
    df["Calendar_Year"] = df.Date.dt.year
    df['Days_In_Year'] = df['Date'].dt.is_leap_year + 365
    df["Day_Of_Year"] = df.Date.dt.dayofyear
    df["Tertiary"] = np.where(df.Date.dt.month < 5, 1, np.where(df.Date.dt.month < 9, 2, 3))
    df["Calendar_Quarter"] = df.Date.dt.quarter
    df["Day_Of_Quarter"] = (df.Date - pd.PeriodIndex(df.Date,freq='Q').start_time).dt.days + 1
    df["Start_Of_Quarter"] = pd.PeriodIndex(df.Date,freq='Q').start_time
    df["End_Of_Quarter"] = pd.PeriodIndex(df.Date,freq='Q').end_time
    df["Days_In_Quarter"] = (df.End_Of_Quarter - df.Start_Of_Quarter).dt.days
    df["Week_Of_Quarter"] = (df.Day_Of_Quarter - 1) // 7 + 1
    df["Month_Of_Quarter"] = (df.Date.dt.month - 1) % 3 + 1
    df["Calendar_Year_Quarter"] = df.Date.dt.year.astype(str) + ' Qtr ' + df.Date.dt.quarter.astype(str).str.zfill(2)
    df["Month"] = df.Date.dt.month
    df["Day_Of_Month"] = df.Date.dt.day
    df["Start_Of_Month"] = pd.PeriodIndex(df.Date,freq='M').start_time
    df["End_Of_Month"] = pd.PeriodIndex(df.Date,freq='M').end_time
    df["Days_In_Month"] = df.Date.dt.days_in_month
    df["Week_Of_Month"] = (df.Day_Of_Month - 1) // 7 + 1
    df["Year_Month"] = df.Date.dt.year.astype(str) + '-' + df.Date.dt.month.astype(str).str.zfill(2)

# ISO
    df["Year"] = df.Date.dt.isocalendar().year.astype(int)
    df["Week_No"] = df.Date.dt.isocalendar().week.astype('int64')
    df["Year_Week"] = df.Date.dt.isocalendar().year.astype(str) + '-' + df.Date.dt.strftime("%V")
    df["Year_Week_Short"] = df.Date.dt.strftime("%y").astype(str) + 'W' + df.Date.dt.strftime("%V")
    df["Year_Week_Date"] = df.Date - df.Date.dt.dayofweek * pd.Timedelta(days=1)
    df["Year_Quarter"] = np.where(df.Date.dt.isocalendar().week.astype('int64') < 14, 1, np.where(df.Date.dt.isocalendar().week.astype('int64') < 27, 2, np.where(df.Date.dt.isocalendar().week.astype('int64') < 40, 3, 4)))
    df["Year_Quarter_Short"] = df.Date.dt.isocalendar().year.astype(str) + '-Q' + df["Year_Quarter"].astype(str)

# MONTH GROUPING
    def calendarGroup(pattern: list, weekCol):
        # Define range of months
        x = list(range(1,13))

        # Define week pattern
        y = pattern * 4

        period = list(np.repeat(x, y))
        wk = [i for i in range(1,53)]

        di_wk = dict(zip(wk, period))

        return weekCol.map(di_wk)

    df["Month_544"] = calendarGroup([5, 4, 4], df["Week_No"]).astype(pd.Int64Dtype())
    df["Month_445"] = calendarGroup([4, 4, 5], df["Week_No"]).astype(pd.Int64Dtype())    
    
    df["Year_544_Month"] = df.Date.dt.year.astype(str) + ' ' + df["Month_544"].astype(str).str.zfill(2)
    df["Year_445_Month"] = df.Date.dt.year.astype(str) + ' ' + df["Month_445"].astype(str).str.zfill(2)

# IDENTIFIERS
    # df['FirstDayOfMonthFlag'] = (df['Date'].dt.is_month_start).astype(int)
    # df['LastDayOfMonthFlag'] = (df['Date'].dt.is_month_end).astype(int)
    # df['IsTodayFlag']=(df['Date'] == pd.Timestamp.today().date()).astype(int)
    df['Is_Leap_Year'] = df['Date'].dt.is_leap_year
    df['Is_Today'] = np.where(df['Date'] == today, True, False)
    df['Is_Weekend'] = np.where(df['Date'].dt.day_of_week > 4, True, False)


# OFFSETS
    df['Next_Day'] = df['Date'] + DateOffset(days=1)
    df['Previous_Day'] = df['Date'] - DateOffset(days=1)
    df['Date_Offset'] = (df['Date'] - today).dt.days
    df["ISO_Week_Offset"] = (df.Date - df.Date.dt.dayofweek * pd.Timedelta(days=1) - today).dt.days // 7 + 1
    df['ISO_Year_Offset'] = df['Date'].dt.isocalendar().year.astype(int) - today.year
    df['Tjek_Date_Name'] = np.where(df['Date'] == today, "Today", df.Date.dt.strftime("%d-%m-%Y %a").astype(str))
    df['Tjek_Week_Name'] = np.where(df['ISO_Week_Offset'] == 0, "Current week", df.Date.dt.strftime("%Y W%V").astype(str))
    NielsenDate = datetime.strptime("2023-12-03", '%Y-%m-%d')
    df["4_Weeks"] = "4 w/e " + (df.Date + (NielsenDate - df.Date).dt.days % 28 * pd.Timedelta(days=1)).dt.strftime("%d/%m/%y").astype(str)
    df["4_W_Date"] = df['4_Weeks'].str[-8:]
    df["4_W_Date"] = pd.to_datetime(df['4_W_Date'], format='%d/%m/%y', errors='coerce')


    return df



StatementMeta(, cf4561ed-bff4-4185-a0fc-ed0a34b91db6, 5, Finished, Available, Finished)

In [4]:
df = date_dimension(start_date, end_date)
# display(df.dtypes)

# In[4]:


(spark.createDataFrame(df)
.write
.mode("overwrite")
.option("overwriteSchema", "true")
.format("delta")
.saveAsTable("date")
)


StatementMeta(, cf4561ed-bff4-4185-a0fc-ed0a34b91db6, 6, Finished, Available, Finished)